## Ensembles of Decision Trees

*Ensembles* are methods that combine multiple machine learning models to create more powerful models.  *Random forests* and *gradient-boosted decision trees* are two ensemble models that have proven to be effective on a wide range of datasets for classification and regression.  Both use decision trees as their building blocks.

### Random Forests

A main drawback of decision trees is that they tend to overfit the training data.  Random forests are one way to address this problem.  

A *random forest* is essentially a collection of decision trees, where each tree is slightly different from the others.  The idea behind random forests is that each tree might do a relatively good job of predicting, but will likely overfit on part of the data.  If we build many trees, all of which work well and overfit in different ways, we can reduce the amount of overfitting by averaging their results.  This reduction of overfitting can be shown using rigorous mathematics.

To implement this strategy, we need to build many decision trees.  Each tree should do an acceptable job of predicting the target, and should be different from the other trees.

Random forests get their name from injecting randomness into the tree building to ensure each tree is different.  There are 2 ways in which the trees in a random forest are randomized:
- By selecting the data points used to build a tree
- By selecting the features in each test split